Print pandas version. IF Pandas Version > 1.0 then 

* from pandas.io.json import json_normalize -> from pandas import json_normalize

In [1]:
import sys
import pandas
print("Python Version : ", sys.version)
print("Pandas Version : ", pandas.__version__)

Python Version :  3.8.6 | packaged by conda-forge | (default, Oct  7 2020, 19:08:05) 
[GCC 7.5.0]
Pandas Version :  1.1.3


In [ ]:
import requests, json

import pandas as pd
from pandas.io.json import json_normalize

class KibanaClient:
    '''This is the Kibana Client.'''
    def __init__(self, hostname, port=5601):
        self.base_url = "http://{0}:{1}".format(hostname, port)
        # https://www.elastic.co/guide/en/kibana/master/using-api.html#api-request-headers
        self.headers = {'kbn-xsrf': 'true', 'Accept': 'application/json', 'Content-Type': 'application/json'}

    def dashboard(self, dashboard, debug=False):
        jsonObj = self.readJsonURL('/api/kibana/dashboards/export?dashboard={0}'.format(dashboard), "", debug)       
        return jsonObj

    def readJsonURL(self, path, body="", debug=False):
        '''Read JSON from URL'''
        url = self.base_url + path
        if (debug):
            print("Read url from :", url)
        content = requests.post(url, headers=self.headers, data=body, allow_redirects=True)
        jsonObj = json.loads(content.content)
        return jsonObj
    
    def indices(self, debug=False):
        jsonObj = self.readProxyURL("GET", "/_cat/indices?format=json", debug)       
        return jsonObj

    def searchTerms(self, index, terms, debug=False):
        query = {
            'term': terms
        }
        return self.search(index, query, debug)
    
    def search(self, index, query, debug=False):
        path = "/{0}/_search".format(index)
        query_dict = {
            'query': query
        }
        query_json = json.dumps(query_dict)
        jsonObj = self.readProxyURL("POST", path, query_json, debug)       
        return jsonObj
    

    def readProxyURL(self, method, path, body="", debug=False):
        return self.readJsonURL('/api/console/proxy?method={0}&path={1}'.format(method, path), body, debug)
